In [ ]:
import sys
import pandas as pd

sys.path.append("../")
from pathlib import Path

In [ ]:
### First, prove a video player can use the granicus url for mp4

In [ ]:
# Cell to retrieve download URL and create a simple player
import asyncio
import os
from IPython.display import HTML, display
from src.granicus import get_video_player

# URL for the Granicus player page
granicus_url = "https://tulsa-ok.granicus.com/MediaPlayer.php?view_id=4&clip_id=6399"
player = await get_video_player(granicus_url)

player_html = f"""
<div style="max-width: 640px; margin: 20px auto; border: 1px solid #ddd; padding: 10px; background: #f5f5f5;">
    <h3 style="margin-top: 0; color: #333;">Direct Video Player</h3>
    
    <video id="granicus-video" controls width="100%" preload="metadata" style="background: #000;">
    <source src="{player.download_url}" type="video/mp4">
    Your browser does not support the video tag.
    </video>
    
    <div style="margin-top: 10px; font-size: 12px; color: #666;">
    <p>Using direct MP4 download URL instead of streaming URL for better playback in notebook.</p>
    </div>
</div>
"""

display(HTML(player_html))



In [ ]:
# Import the necessary modules
from pathlib import Path
import json
from IPython.display import HTML, display

# Import from the new subtitles module
from src.subtitles import create_track, load_transcript
from src.models.subtitles import TrackFormat

# Path to the transcript file
transcript_file = Path(
    "../data/transcripts/regular_council_meeting___2025_02_26.diarized.json"
)

# Create VTT track
vtt_track = create_track(
    transcript_data=transcript_file,
    track_format='vtt',
    max_duration=5.0,
    include_speaker_prefix=False,
)

# Get the VTT content
vtt_content = vtt_track.content()

# For demonstration, print a few lines of the VTT content
print(vtt_content.split("\n\n")[:5])

In [ ]:
# Cell to create a video player with embedded VTT subtitles
import base64
from IPython.display import HTML, display
from src.granicus import get_video_player

# URL for the Granicus player page
granicus_url = "https://tulsa-ok.granicus.com/MediaPlayer.php?view_id=4&clip_id=6399"
player = await get_video_player(granicus_url)
vtt_data_url = f"data:text/vtt;charset=utf-8;base64,{base64.b64encode(vtt_content.encode('utf-8')).decode('ascii')}"

# Generate VTT subtitles (using the code from the previous cell if needed)
# Display the sample VTT subtitles in the video player
player_html = f"""
<div style="max-width: 640px; margin: 20px auto; border: 1px solid #ddd; padding: 10px; background: #f5f5f5;">
    <h3 style="margin-top: 0; color: #333;">Video Player with Embedded VTT</h3>
    
    <video id="video-with-data-vtt" controls width="100%" preload="metadata" style="background: #000;">
    <source src="{player.download_url}" type="video/mp4">
    <track kind="subtitles" srclang="en" label="English" default src="{vtt_data_url}">
    Your browser does not support the video tag.
    </video>
    
    <div style="margin-top: 10px; font-size: 12px; color: #666;">
    <p>Video player with subtitles generated using the subtitles module.</p>
    </div>
</div>
"""

display(HTML(player_html))